In [ ]:
import pandas as pd
import pulp



In [3]:


def solve_project_plan(df, case="expected"):
    # LP Model
    lp_problem = pulp.LpProblem("Project_Planning", pulp.LpMinimize)
    
    # Decision vars
    start_times = pulp.LpVariable.dicts("start_time", df["taskID"], 0, None)
    
    # Objective function
    project_end = pulp.LpVariable("project_end", 0, None)
    lp_problem += project_end, "Minimize_Project_Duration"
    
    # adding constraints
    for index, row in df.iterrows():
        task = row["taskID"]
        
        # duration of cases
        if case == "best":
            duration = row["bestCaseHours"]
        elif case == "worst":
            duration = row["worstCaseHours"]
        else:
            duration = row["expectedHours"]
        
        predecessors = str(row["predecessorTaskIDs"]).split()
        
        # task duration constraint
        lp_problem += start_times[task] + duration <= project_end
        
        # task dependencies constraints
        for predecessor in predecessors:
            lp_problem += start_times[task] >= start_times[predecessor] + duration
    
    # Solver
    lp_problem.solve()
    
    # output prep
    results = {
        "status": pulp.LpStatus[lp_problem.status],
        "optimal_duration": project_end.varValue,
        "task_start_times": {task: var.varValue for task, var in start_times.items()}
    }
    
    return results

In [ ]:
xcases = ["best", "expected", "worst"]
for case in cases:
    result = solve_project_plan(project_plan_df, case)
    print(f"Results for {case} case:")
    print(f"Optimal project duration: {result['optimal_duration']} hours")
    print("Start times for tasks:")
    for task, start_time in result["task_start_times"].items():
        print(f"Task {task}: {start_time} hours")
    print("\n")

In [4]:
lp_problem = pulp.LpProblem("Project_Planning", pulp.LpMinimize)

# Decision Variables
start_times = pulp.LpVariable.dicts("start_time", project_plan_df["taskID"], 0, None)

# Obj Function
project_end = pulp.LpVariable("project_end", 0, None)

lp_problem += project_end, "Minimize_Project_Duration"

# Next the constraints

for index, row in project_plan_df.iterrows():
    task = row["taskID"]
    duration = row["expectedHours"]
    predecessors = str(row["predecessorTaskIDs"]).split()
    
    lp_problem += start_times[task] + duration <= project_end
    
    for predecessor in predecessors:
        lp_problem += start_times[task] >= start_times[predecessor] + duration

NameError: name 'project_plan_df' is not defined